# Scraping de Propiedades en Infocasas

Este proyecto realiza un web scraping de las propiedades de alquiler en Montevideo desde el sitio web [Infocasas](https://www.infocasas.com.uy). El objetivo es extraer información sobre apartamentos disponibles para alquilar, como título, URL, imagen, precio y ubicación, y guardarla en un archivo Excel.

## Descripción

El script recorre varias páginas de la sección de alquileres de Infocasas, extrayendo datos clave de cada propiedad listada. Los datos extraídos incluyen:

- **Título del proyecto**: El nombre o título del apartamento.
- **URL**: El enlace directo a la página del apartamento.
- **Imagen**: URL de la imagen principal del apartamento.
- **Precio**: El precio de alquiler de la propiedad.
- **Ubicación**: La ubicación del apartamento en Montevideo.
- **Tipo**: El tipo de propiedad (en este caso, "Alquiler UY").

## Requisitos

- `requests`: Para realizar las solicitudes HTTP al sitio web.
- `BeautifulSoup`: Para parsear y extraer datos del HTML.
- `pandas`: Para almacenar y manipular los datos.
- `openpyxl`: Para guardar los datos en un archivo Excel.

Puedes instalar las dependencias con:

```bash
pip install requests beautifulsoup4 pandas openpyxl

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os


os.makedirs("Resultados", exist_ok=True)

# Diccionario con las URLs y el número de páginas a recorrer
# CATEGORIES = {
#     "En Pozo": ("https://www.infocasas.com.uy/venta/apartamentos/montevideo/en-pozo?page={}", 20),
#     "En Construcción": ("https://www.infocasas.com.uy/venta/apartamentos/montevideo/en-construccion?page={}", 10),
#     "A Estrenar": ("https://www.infocasas.com.uy/venta/apartamentos/montevideo/a-estrenar?page={}", 25),
# }

CATEGORIES = {
    "Alquiler UY": ("https://www.infocasas.com.uy/alquiler/apartamentos", 45), #Esto es relativo, se puede investigar antes las paginas
}


HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

data = []

for category, (base_url, total_pages) in CATEGORIES.items():
    print(f"Scrapeando '{category}'")

    for page in range(1, total_pages + 1):
        url = base_url.format(page)
        # print(f"  > Página {page}...")

        response = requests.get(url, headers=HEADERS)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")

            properties = soup.find_all("div", class_="listingCard")

            if not properties:
                print(f"  No se encontraron más propiedades en la página {page}. Deteniendo")
                break

            for prop in properties:
                
                title_tag = prop.find("a", class_="lc-cardCover")
                title = title_tag.get("title") if title_tag else "N/A"

                project_url = title_tag.get("href") if title_tag else "#"
                full_url = f"https://www.infocasas.com.uy{project_url}"

                img_tag = prop.find("img", alt="Image")
                img_url = img_tag["src"] if img_tag else "N/A"

                price_tag = prop.find("div", class_="lc-price")
                price = price_tag.text.strip() if price_tag else "N/A"

                location_tag = prop.find("strong", class_="lc-location")
                location = location_tag.text.strip() if location_tag else "N/A"

                data.append([title, full_url, img_url, price, location, category])

        else:
            print(f" Error en la página {page}, código {response.status_code}. Saltando")

df = pd.DataFrame(data, columns=["Proyecto", "URL", "Imagen", "Precio", "Ubicación", "Tipo"])


file_path = "Resultados/InfoCasas_Alquiler_Completo.xlsx"
# file_path = "Resultados/InfoCasas_Completo.xlsx"
df.to_excel(file_path, index=False)

print(f"\n Cargado con éxito en {file_path}")

In [ ]:
# df_datos = pd.read_excel("C:/Users/ASUS/OneDrive/Cosas/Desktop/Escritorio Trabajo/Inmovin/WS/Resultados/InfoCasas_Completo.xlsx")

df_datos = df

df_datos.head(5)

In [ ]:
def obtener_datos_propiedad(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error al obtener la página: {e}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    propiedad_info = {}

    try:

        dormitorios_tag = soup.find("span", class_="ant-typography ant-typography-ellipsis", string=lambda text: text and "Dorm." in text)
        precio_tag = soup.find("span", class_="ant-typography price")
        gastos_comunes_tag = soup.find("span", class_="ant-typography ant-typography-secondary commonExpenses")
        
        propiedad_info["Dormitorios"] = dormitorios_tag.get_text(strip=True) if dormitorios_tag else "Sin dato"
        propiedad_info["Precio"] = precio_tag.find("strong").get_text(strip=True) if precio_tag and precio_tag.find("strong") else "Sin dato"
        propiedad_info["Gastos Comunes"] = gastos_comunes_tag.get_text(strip=True) if gastos_comunes_tag else "Sin dato"
        

        spans = soup.find_all("span", class_="ant-typography ant-typography-ellipsis")
        for span in spans:
            text = span.get_text(strip=True)
            if "Baño" in text:
                propiedad_info["Baños"] = text
            elif "m²" in text:
                propiedad_info["M²"] = text
        
        propiedad_info.setdefault("Baños", "Sin dato")
        propiedad_info.setdefault("M²", "Sin dato")
    
    except AttributeError as e:
        print(f"Error al extraer datos: {e}")
    
    return propiedad_info

# Aca es un ejemplo de filtro en el df
df_a_estrenar = df_datos[df_datos["Tipo"] == "A Estrenar"].copy()

nuevos_datos = []
for index, row in df_a_estrenar.iterrows():
    url = row["URL"]
    print(f"Obteniendo datos para: {url}")
    nuevos_datos.append(obtener_datos_propiedad(url) or {"Dormitorios": "Sin dato", "Precio": "Sin dato", "Gastos Comunes": "Sin dato", "Baños": "Sin dato", "M²": "Sin dato"})

if nuevos_datos:
    df_nuevos_datos = pd.DataFrame(nuevos_datos)
    df_a_estrenar = pd.concat([df_a_estrenar.reset_index(drop=True), df_nuevos_datos], axis=1)

df_a_estrenar.to_excel("Resultados/A_Estrenar_Completo.xlsx", index=False, engine="xlsxwriter")

print("\n Datos guardados en Resultados/A_Estrenar_Completo.xlsx")